<a href="https://colab.research.google.com/github/xdderekchen/ML/blob/master/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

 * sklearn Linear Regressor
 * sm
 * SPARK ML
 * TensorFlow
   * Single-layers
   * Multi-layers

## DataSet Preparation
 using data from http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data

In [82]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
                                                     
dataset_path = keras.utils.get_file("auto-mpg.data2", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
column_names = ["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "model year", "origin"]

raw_dataset = pd.read_csv(dataset_path, names=column_names, na_values="?", comment="\t", sep=" ", skipinitialspace=True)
dataset = raw_dataset.dropna().copy()

origin = dataset.pop("origin")

dataset["USA"]    = (origin == 1) * 1.0
dataset["Europe"] = (origin == 2) * 1.0
dataset["Japan"]  = (origin == 3) * 1.0

XNames = [x for x in dataset.columns if x != "mpg"]
Y = dataset["mpg"]           #series
X = dataset[XNames]

#Split dataset as train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)

#Scale X values
from sklearn.preprocessing import  StandardScaler
scaler = StandardScaler( )
X_train = scaler.fit_transform( X_train )
X_test = scaler.fit_transform ( X_test )

#to report result
def linear_Reg_report(dataset_type, predict_Y, actual_Y):
    rmse = (np.sqrt(mean_squared_error(actual_Y, predict_Y)))
    r2 = r2_score(actual_Y, predict_Y)

    print("\nThe model performance for training set")
    print("--------------------------------------")
    print('RMSE is {}'.format(rmse))
    print('R2 score is {}'.format(r2))
    print("\n")

2.1.0


## LinearRegression from sklearn

In [85]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,  r2_score

lin_model = LinearRegression()

lin_model.fit( X_train, Y_train )

# model evaluation for training set
y_train_predict = lin_model.predict(X_train)
# model evaluation for testing set
y_test_predict = lin_model.predict(X_test)

coefDataFrame = pd.DataFrame({'features':         ["intercept"], 
                              'estimatedCoeffs':  lin_model.intercept_.flatten()
                              }
                             )
coefDataFrame = coefDataFrame.append(pd.DataFrame( zip(XNames, lin_model.coef_.flatten()), columns=['features', 'estimatedCoeffs']), ignore_index=True)
print("regression cooefficients:\n")
print(coefDataFrame)

linear_Reg_report("training", y_train_predict, Y_train)
linear_Reg_report("testing",  y_test_predict, Y_test)



regression cooefficients:

       features  estimatedCoeffs
0     intercept     2.348041e+01
1     cylinders    -5.608855e-01
2  displacement     2.294041e+00
3    horsepower    -4.852167e-01
4        weight    -5.867469e+00
5  acceleration     3.628640e-01
6    model year     2.788355e+00
7           USA    -1.499224e+14
8        Europe    -1.160200e+14
9         Japan    -1.254847e+14

The model performance for training set
--------------------------------------
RMSE is 3.3076473263049575
R2 score is 0.818879307342756



The model performance for training set
--------------------------------------
RMSE is 9936368359114.354
R2 score is -1.5884629721673072e+24


